In [16]:
import os
from PIL import Image
import argparse

def rename_photos_in_folders(root_dir):
    """
    Переименовывает все фото в подпапках в формат 01.jpg, 02.jpg, ..., 10.jpg
    """
    for subdir, _, files in os.walk(root_dir):
        image_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif'))]
        if not image_files:
            continue
            
        image_files.sort()
        prefix = subdir.split("/")[-1]
        for i, filename in enumerate(image_files, 1):
            old_path = os.path.join(subdir, filename)

            ext = os.path.splitext(filename)[1].lower()

            new_name = f"{prefix}_{i:02d}{ext}"
            new_path = os.path.join(subdir, new_name)

            try:
                os.rename(old_path, new_path)
                print(f"Переименовано: {filename} -> {new_name}")
            except Exception as e:
                print(f"Ошибка при переименовании {filename}: {e}")

rename_photos_in_folders('./for_recognize/dataset/')


Обрабатываю папку: ./for_recognize/dataset/Albert Einstein
Найдено 10 изображений
Переименовано: Albert Einstein_01.png -> Albert Einstein_01.png
Переименовано: Albert Einstein_02.png -> Albert Einstein_02.png
Переименовано: Albert Einstein_03.png -> Albert Einstein_03.png
Переименовано: Albert Einstein_04.png -> Albert Einstein_04.png
Переименовано: Albert Einstein_05.png -> Albert Einstein_05.png
Переименовано: Albert Einstein_06.png -> Albert Einstein_06.png
Переименовано: Albert Einstein_07.png -> Albert Einstein_07.png
Переименовано: Albert Einstein_08.png -> Albert Einstein_08.png
Переименовано: Albert Einstein_09.png -> Albert Einstein_09.png
Переименовано: Albert Einstein_10.png -> Albert Einstein_10.png

Обрабатываю папку: ./for_recognize/dataset/Alla Pugacheva
Найдено 10 изображений
Переименовано: Alla Pugacheva_01.png -> Alla Pugacheva_01.png
Переименовано: Alla Pugacheva_02.png -> Alla Pugacheva_02.png
Переименовано: Alla Pugacheva_03.png -> Alla Pugacheva_03.png
Переимено

Переименовано: Tom Hardy_11.png -> Tom Hardy_11.png
Переименовано: Tom Hardy_12.png -> Tom Hardy_12.png

Обрабатываю папку: ./for_recognize/dataset/Uma Thurman
Найдено 11 изображений
Переименовано: Uma Thurman_01.png -> Uma Thurman_01.png
Переименовано: Uma Thurman_02.png -> Uma Thurman_02.png
Переименовано: Uma Thurman_03.png -> Uma Thurman_03.png
Переименовано: Uma Thurman_04.png -> Uma Thurman_04.png
Переименовано: Uma Thurman_05.png -> Uma Thurman_05.png
Переименовано: Uma Thurman_06.png -> Uma Thurman_06.png
Переименовано: Uma Thurman_07.png -> Uma Thurman_07.png
Переименовано: Uma Thurman_08.png -> Uma Thurman_08.png
Переименовано: Uma Thurman_09.png -> Uma Thurman_09.png
Переименовано: Uma Thurman_10.png -> Uma Thurman_10.png
Переименовано: Uma Thurman_11.png -> Uma Thurman_11.png

Обрабатываю папку: ./for_recognize/dataset/Viktor Tsoi
Найдено 10 изображений
Переименовано: Viktor Tsoi_01.png -> Viktor Tsoi_01.png
Переименовано: Viktor Tsoi_02.png -> Viktor Tsoi_02.png
Переименов

In [17]:
import os
import json
from glob import glob

def generate_label_studio_import(dataset_path, output_file="label_studio_import.json"):
    tasks = []
    
    for person_folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, person_folder)
        if not os.path.isdir(folder_path):
            continue
            
        # Получаем все изображения в папке (сортированные по имени)
        image_files = sorted(glob(os.path.join(folder_path, "*.jpg")) + 
                            glob(os.path.join(folder_path, "*.png")))
        
        for img_path in image_files:
            # Создаем запись для Label Studio
            task = {
                "data": {

                    "image": f"/data/local-files/?d={img_path}",  # Путь будет настроен в LS
                    "folder": person_folder  # Название папки как метка класса
                }
            }
            tasks.append(task)
    
    # Сохраняем в JSON
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(tasks, f, ensure_ascii=False, indent=2)
    
    print(f"Сгенерировано {len(tasks)} задач. Файл сохранен как {output_file}")

# Пример использования
generate_label_studio_import("for_recognize/dataset/")

Сгенерировано 450 задач. Файл сохранен как label_studio_import.json


In [ ]:
import cv2
from ultralytics import YOLO
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Инициализация модели YOLO для детекции лиц
model = YOLO('best(1).pt')  # Автоматически скачает модель при первом запуске

# Путь к папке с изображениями
image_dir = Path(r"../label-studio/label-studio-files/dataset/")

# Функция для отображения изображения с bounding boxes
def show_image_with_boxes(img_path, results):
    # Загрузка оригинального изображения
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Рисуем bounding boxes на изображении
    for result in results:
        for box in result.boxes:
            # Координаты bounding box
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            
            # Рисуем прямоугольник
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            
            # Добавляем текст с уверенностью
            conf = box.conf.item()
            cv2.putText(img, f"{conf:.2f}", (x1, y1-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    
    # Отображаем изображение
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Detected faces in {img_path.name}")
    plt.show()

image_extensions = ['.jpg', '.jpeg', '.png']

for img_path in image_dir.glob('**/*'):
    if img_path.suffix.lower() in image_extensions: # Можно добавить другие форматы: *.jpeg, *.png
    # Выполнение детекции без сохранения
        results = model.predict(
            source=str(img_path),
            save=False,  # Не сохраняем на диск
            conf=0.5,    # Порог уверенности
            iou=0.5      # Порог IoU для NMS
        )

        # Отображаем результаты
        show_image_with_boxes(img_path, results)

        # Выводим информацию о детекции
        print(f"Файл: {img_path}")
        print("Обнаруженные лица:")
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                conf = box.conf.item()
                print(f"- Координаты: [{x1:.1f}, {y1:.1f}, {x2:.1f}, {y2:.1f}], Уверенность: {conf:.2f}")
        print("\n" + "="*50 + "\n")

print("Обработка завершена")

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.models as models

In [9]:
data_dir = '../label-studio/label-studio-files/cropped_dataset/'
model_save_path = '../face_recognition_resnet_new1111.pth'
labels_file = '../labels_organized.txt'
num_epochs = 10
batch_size = 32
learning_rate = 0.001
num_workers = 2  

In [10]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

image_dataset = datasets.ImageFolder(data_dir, transform)
dataloader = DataLoader(image_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
class_names = image_dataset.classes
num_classes = len(class_names)
print(class_names)
print(num_classes)

['Albert Einstein', 'Alla Pugacheva', 'Angelina Jolie', 'Arnold Schwarzenegger', 'Boris Yeltsin', 'Brad Pitt', 'Britney Spears', 'Demi Moore', 'Donald Trump', 'Elon Musk', 'Eminem', 'Freddie Mercury', 'Guy Ritchie', 'Ivan Basalaev', 'Jason Statham', 'Jennifer Lopez', 'Jim Carrey', 'Johnny Depp', 'Lady Gaga', 'Leonardo DiCaprio', 'Madonna', 'Marilyn Monroe', 'Mark Zuckerberg', 'Martin Scorsese', 'Matthew McConaughey', 'Michael Jackson', 'Mike Tyson', 'Monica Bellucci', 'Quentin Tarantino', 'Robert Downey Jr', 'Salma Hayek', 'Scarlett Johansson', 'Steve Jobs', 'Sylvester Stallone', 'Tom Cruise', 'Tom Hardy', 'Uma Thurman', 'Viktor Tsoi', 'Vin Diesel', 'Vladimir Vysotsky', 'Will Smith', 'Yuri Nikulin']
42


In [11]:
with open(labels_file, 'w') as f:
    str2write = "\n".join(class_names)
    f.write(str2write)
print(f"Файл labels сохранен в: {labels_file}")

Файл labels сохранен в: labels_organized.txt


In [12]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

cuda


In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
# --- Обучение модели ---
for epoch in range(num_epochs):
    print(f'Эпоха {epoch+1}/{num_epochs}')
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(image_dataset)
    print(f'  Потери: {epoch_loss:.4f}')

print(f'Обучение завершено. Сохранение модели в: {model_save_path}')
torch.save(model.state_dict(), model_save_path)

Эпоха 1/10
  Потери: 3.0026
Эпоха 2/10
  Потери: 0.9612
Эпоха 3/10
  Потери: 0.2443
Эпоха 4/10
  Потери: 0.0868
Эпоха 5/10
  Потери: 0.0420
Эпоха 6/10
  Потери: 0.0399
Эпоха 7/10
  Потери: 0.0331
Эпоха 8/10
  Потери: 0.0320
Эпоха 9/10
  Потери: 0.0348
Эпоха 10/10
  Потери: 0.0330
Обучение завершено. Сохранение модели в: face_recognition_resnet_new1111.pth


In [21]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

# --- Настройки ---
model_path = '../face_recognition_resnet_new1111.pth'
labels_path = '../labels_organized.txt'
name = 'Ivan Basalaev'
test_image_path = f'./label-studio/label-studio-files/cropped_dataset/{name}/{name}_01.jpg' 
detection_threshold = 0.5  

# --- Загрузка меток классов ---
with open(labels_path, 'r') as f:
    class_names = [line.strip() for line in f.readlines()]
num_classes = len(class_names)

# --- Загрузка предобученной модели ResNet18 и изменение FC слоя ---
model = models.resnet18(pretrained=False)  
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Загрузка весов обученной модели
model.load_state_dict(torch.load(model_path))
model.eval()  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# --- Преобразования для тестового изображения (должны совпадать с тренировочными) ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# --- Функция для распознавания лица на изображении ---
def recognize_face(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        img_tensor = transform(img).unsqueeze(0).to(device)  

        with torch.no_grad():
            output = model(img_tensor)
            probabilities = torch.softmax(output, dim=1)
            _, predicted_idx = torch.max(probabilities, 1)
            predicted_label = class_names[predicted_idx[0]]
            confidence = probabilities[0, predicted_idx[0]].item() * 100

        return predicted_label, confidence
    except FileNotFoundError:
        return "Ошибка: изображение не найдено", 0
    except Exception as e:
        return f"Произошла ошибка: {e}", 0

# --- Распознавание лица (без детекции YOLOv5) ---
predicted_name, confidence = recognize_face(test_image_path)
print(f"Распознано: {predicted_name} (уверенность: {confidence:.2f}%)")

# --- Распознавание лица с предварительной детекцией YOLOv5  ---
def detect_faces_yolo(image_path, model_yolo, conf_thres=0.5):
    img = Image.open(image_path).convert('RGB')
    results = model_yolo(img)
    faces = results.xyxy[0][:, :4] 
    confidences = results.xyxy[0][:, 4]
    face_coords = [(int(x1), int(y1), int(x2), int(y2)) for *xyxy, conf in zip(*faces.T, confidences) if conf >= conf_thres]
    return face_coords

# для теста на нескольких лицах
use_yolo_detection = False  

if use_yolo_detection:
    yolo_model_path = '/Users/dariamotronenko/Desktop/Android/yolov5/runs/train/exp/weights/best.pt' 
    model_yolo = torch.hub.load('ultralytics/yolov5', 'custom', path=yolo_model_path)
    model_yolo.eval()

    test_image_with_faces = 'path/to/your/image_with_multiple_faces.jpg' 
    detected_faces = detect_faces_yolo(test_image_with_faces, model_yolo, conf_thres=detection_threshold)

    for i, (x1, y1, x2, y2) in enumerate(detected_faces):
        face_img = Image.open(test_image_with_faces).convert('RGB').crop((x1, y1, x2, y2))
        face_tensor = transform(face_img).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(face_tensor)
            probabilities = torch.softmax(output, dim=1)
            _, predicted_idx = torch.max(probabilities, 1)
            predicted_label = class_names[predicted_idx[0]]
            confidence = probabilities[0, predicted_idx[0]].item() * 100
            print(f"Обнаружено лицо {i+1}: Распознано как {predicted_label} (уверенность: {confidence:.2f}%)")

Распознано: Ivan Basalaev (уверенность: 99.17%)


In [1]:
import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import shutil
import numpy as np

# Пути
DATASET_DIR = "../label-studio/label-studio-files/cropped_dataset/"
MODEL_SAVE_PATH = "face_recognition_resnet.pth"
CLASS_NAMES_PATH = "../class_names.txt"
BATCH_SIZE = 8
EPOCHS = 50
LR = 0.001

# Подготовка данных
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class FaceDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image)

        return image, label

def create_dataloaders():
    image_paths = []
    labels = []
    class_names = []

    for root, dirs, files in os.walk(DATASET_DIR):
        for dir_name in dirs:
            person_dir = os.path.join(root, dir_name)
            img_files = [os.path.join(person_dir, f) for f in os.listdir(person_dir) if f.endswith(".jpg")]
            image_paths.extend(img_files)
            labels.extend([len(class_names)] * len(img_files))
            class_names.append(dir_name)

    # Сохраняем имена классов
    with open(CLASS_NAMES_PATH, 'w') as f:
        f.write('\n'.join(class_names))

    # Разделение на train/test
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        image_paths, labels, test_size=0.2, random_state=42, stratify=labels
    )

    train_dataset = FaceDataset(train_paths, train_labels, transform=train_transform)
    test_dataset = FaceDataset(test_paths, test_labels, transform=test_transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

    return train_loader, test_loader, len(class_names)

# Обучение модели
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(train_loader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    acc = 100. * correct / total
    print(f"Train Loss: {running_loss:.4f}, Accuracy: {acc:.2f}%")

# Тестирование модели
def test_model(model, test_loader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    acc = 100. * correct / total
    print(f"Test Accuracy: {acc:.2f}%")


train_loader, test_loader, num_classes = create_dataloaders()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка предобученной модели ResNet18
model = models.resnet18(weights="IMAGENET1K_V1")
for param in model.parameters():
    param.requires_grad = False  # Замораживаем веса

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)  # Изменяем голову под нашу задачу
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=LR)

print(f"\nНачинаем обучение на {num_classes} классах...\n")

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    train_model(model, train_loader, criterion, optimizer, device)
    test_model(model, test_loader, criterion, device)

# Сохранение модели
torch.save(model.state_dict(), MODEL_SAVE_PATH)
print(f"\nМодель сохранена в {MODEL_SAVE_PATH}")


/home/basalegend/PycharmProjects/Android/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0



Начинаем обучение на 42 классах...


Epoch 1/50


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:06<00:00,  6.67it/s]


Train Loss: 188.8110, Accuracy: 3.06%
Test Accuracy: 5.56%

Epoch 2/50


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:07<00:00,  6.42it/s]


...

Train Loss: 18.3148, Accuracy: 92.20%
Test Accuracy: 65.56%

Epoch 49/50


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:07<00:00,  6.42it/s]


Train Loss: 20.2704, Accuracy: 92.48%
Test Accuracy: 64.44%

Epoch 50/50


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:07<00:00,  6.25it/s]


Train Loss: 17.6308, Accuracy: 93.31%
Test Accuracy: 66.67%

Модель сохранена в resnet18_face111.pth
